# Get all relevant submissions

Author: Junita Sirait

I am tired of parsing the whole 6 months of data each time I need additional informations from the JSON files. Here I will master the art of `ndjson` dump and load, so I can just process these files in the future.

Table of contents:
1. [Reading the original massive files](#sub1)

In [1]:
# !pip install ndjson

You should consider upgrading via the 'C:\Users\User200803\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import ndjson

import json
import pandas as pd
import zstandard as zstd
import io

from collections import defaultdict, Counter
from urllib.parse import urlparse
import re
import datetime, time
import tldextract

## Reading the original massive files

In [16]:
with open("D:\\Wellesley\\F21\\thesis\\data\\gmm_intersection.json", "r") as infile:
    gmm_news_sources = json.load(infile)

In [3]:
dctx = zstd.ZstdDecompressor(max_window_size=2147483648)

In [4]:
def findURLs(phrase):
    regex = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)')
    url = re.findall(regex, phrase)     
    return [x[0] for x in url]

In [7]:
def get_hostname(url, uri_type='both'):
    """Get the host name from the url"""
    hostnames = set()
    extracted = tldextract.extract(url)
    subdomain, domain, suffix = extracted
    # add both versions of domain.suffix and subdomain.domain.suffix
    full = ""
    # with subdomain
    if len(subdomain) > 0 and len(suffix) > 0:
        #print(f"{subdomain}.{domain}.{suffix}")
        full = f"{subdomain}.{domain}.{suffix}"
        if len(full) > 0:
            hostnames.add(full[4:].strip('/')) if full.startswith("www.") else hostnames.add(full.strip('/'))
    # without subdomain
    full = f"{domain}.{suffix}"
    if len(full) > 0 and len(suffix) > 0:
        hostnames.add(full[4:].strip('/')) if full.startswith("www.") else hostnames.add(full.strip('/'))
    return hostnames

In [5]:
zst_files = ["RS_2021-01.zst", "RS_2021-02.zst", "RS_2021-03.zst", "RS_2021-04.zst", "RS_2021-05.zst", "RS_2021-06.zst"]
zst_filepath = "E:/thesis_data/"

In [30]:
print("start time:", datetime.datetime.now())


for zst_file in zst_files:
    counter = 0
    added = 0
    month = zst_file[8:10]
    with open ("E:/relevant_posts_{}.ndjson".format(month), "a", encoding="utf-8") as ndjfile:
        writer = ndjson.writer(ndjfile, ensure_ascii=False)
        print("***** Start processing for {} *****".format(zst_file))
        with open(zst_filepath+zst_file, 'rb') as ifh:
            with dctx.stream_reader(ifh, read_size=2) as reader:
                text_stream = io.TextIOWrapper(reader, encoding='utf-8')
                for d in text_stream:
                    line = json.loads(d)
                    URLs = findURLs(line['url']) + findURLs(line['selftext'])                
                    hostnames = [get_hostname(url) for url in URLs]
                    URLs = [element for sub in hostnames for element in sub]
                    # print(URLs)
                    for url in URLs:
                        if url in gmm_news_sources:
                            writer.writerow(line)
                            added += 1
                            break # write this post only once
                    url_of_our_ns = False
                    counter += 1
                    if counter%500000 == 0: 
                        print("processed {}; added {}; \t by {}".format(counter, added, str(datetime.datetime.now())[11:19]))
        
        print("****************************************** Summary ******************************************")
        print(f"There are {counter} total posts in {month}, and {added} of them have urls to our news sources.")
        
        print("----------------------------------------------------------------------------------------")
        print("-------------------------------- Done processing for {} --------------------------------".format(zst_file))
        print("----------------------------------------------------------------------------------------")
                
print("finish time:", datetime.datetime.now())

start time: 2022-02-28 20:38:24.385680
***** Start processing for RS_2021-01.zst *****
processed 500000; added 16805; 	 by 20:39:48
processed 1000000; added 31347; 	 by 20:41:08
processed 1500000; added 46675; 	 by 20:42:49
processed 2000000; added 59251; 	 by 20:44:13
processed 2500000; added 74564; 	 by 20:45:36
processed 3000000; added 88358; 	 by 20:46:57
processed 3500000; added 106606; 	 by 20:48:21
processed 4000000; added 123104; 	 by 20:49:43
processed 4500000; added 141057; 	 by 20:51:05
processed 5000000; added 158496; 	 by 20:52:28
processed 5500000; added 177073; 	 by 20:53:50
processed 6000000; added 197167; 	 by 20:55:11
processed 6500000; added 217261; 	 by 20:56:29
processed 7000000; added 238367; 	 by 20:57:49
processed 7500000; added 258210; 	 by 20:59:05
processed 8000000; added 279678; 	 by 21:00:29
processed 8500000; added 298561; 	 by 21:01:49
processed 9000000; added 315467; 	 by 21:03:08
processed 9500000; added 330722; 	 by 21:04:24
processed 10000000; added 3

KeyboardInterrupt: 

In [31]:
1137576/32704571

0.03478339465146936

In [26]:
"wikipedia.org" in gmm_news_sources

True

In [18]:
counter

235400